In [2]:
import requests
from bs4 import BeautifulSoup



In [3]:
# URL of the webpage to scrape
#url = "https://solardiscount.at/collections/pv-komplettanlage?sort_by=best-selling"
url = "https://solardiscount.at/collections/solarmodule?sort_by=best-selling"

# Download the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
    print("Webpage downloaded successfully")
else:
    print(f"Failed to download webpage. Status code: {response.status_code}")


Webpage downloaded successfully


In [4]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Pretty print the HTML content (first 2000 characters to avoid too much output)
print(soup.prettify()[:2000])


<!DOCTYPE html>
<html class="no-js" dir="ltr" lang="de">
 <head>
  <script src="https://app.shoplytics.de/scripts/QHkz9fsZMsg0F4QZNnKIDAk3eyVYhwxiXGcXHzL8W2EUhr7j0MvdB6TW5S1RjOJ3.js">
  </script>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <title>
   PV-Anlagen mit Speicher als Komplettpaket zum TOP-Preis – Solardiscount.at
  </title>
  <link href="https://solardiscount.at/collections/pv-komplettanlage" rel="canonical"/>
  <link href="//solardiscount.at/cdn/shop/files/S._Solarenergie_zu_Discountpreisen.png?crop=center&amp;height=48&amp;v=1704352229&amp;width=48" rel="icon" type="image/png"/>
  <meta content="Wir stellen alles zusammen, was du für deine Solaranlage brauchst. Kaufe jetzt Solarmodule, Wechselrichter und Speicher zum unschlagbaren Set-Preis." name="description"/>
  <meta content="Solardiscount.at" property="og:site_name"/>
  <meta content="https://solardiscount.at/collections/pv-komplettanlage" property="og:url"/>
  <m

In [5]:
# Print the entire HTML content
print(html_content.decode('utf-8'))


<!doctype html>
<html class="no-js" lang="de" dir="ltr">
<head>
  <script src="https://app.shoplytics.de/scripts/QHkz9fsZMsg0F4QZNnKIDAk3eyVYhwxiXGcXHzL8W2EUhr7j0MvdB6TW5S1RjOJ3.js"></script><meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>PV-Anlagen mit Speicher als Komplettpaket zum TOP-Preis &ndash; Solardiscount.at</title><link rel="canonical" href="https://solardiscount.at/collections/pv-komplettanlage"><link rel="icon" href="//solardiscount.at/cdn/shop/files/S._Solarenergie_zu_Discountpreisen.png?crop=center&height=48&v=1704352229&width=48" type="image/png"><meta name="description" content="Wir stellen alles zusammen, was du für deine Solaranlage brauchst. Kaufe jetzt Solarmodule, Wechselrichter und Speicher zum unschlagbaren Set-Preis."><meta property="og:site_name" content="Solardiscount.at">
<meta property="og:url" content="https://solardiscount.at/collections/pv-komplettanlage">
<meta property="og:title" content="PV-Anlagen mit

In [ ]:
# Find all product elements (assuming they are in a container with a specific class)
product_containers = soup.find_all(class_='product-grid-item')

# Pretty print the HTML content of the first product container (for example)
if product_containers:
    print(product_containers[0].prettify())
else:
    print("No product containers found")
